# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [19]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [7]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [12]:
system_prompt = "You are a Technical expert that will be my coding assistant. I am going to ask you some questions, \
mainly related with software development and mainly in Python. Respond in markdown and providing code snippets if possible"

user_prompt = question + ". Respond in markdown and providing code snippets if possible"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [11]:
# Get gpt-4o-mini to answer, with streaming
def ask_gpt():
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages,
        stream = True
    )

    response = ""
    display_handle = display(Markdown(""), display_id = True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id = display_handle.display_id)

In [9]:
ask_gpt()

The code snippet you provided is using a generator expression alongside a set comprehension in Python. Let's break down what each part does and why it's structured this way.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


1. **Set Comprehension**: 
   - The part `{book.get("author") for book in books if book.get("author")}` is a set comprehension that iterates over `books`, which is assumed to be a list (or any iterable) of dictionaries.
   - For each `book` in `books`, it retrieves the value associated with the "author" key using `book.get("author")`.
   - The `if book.get("author")` condition filters out any books that do not have an "author" key or where the value of the "author" key is `None`, empty, or other falsy values. It ensures that only valid author values are included.
   - The result is a set of unique authors since sets automatically handle duplicate entries.

2. **Yield From**:
   - The `yield from` expression is used inside a generator function. It effectively delegates the yielding of multiple values from an iterable (in this case, the set of authors).
   - This allows the generator to yield each author one at a time as if they were yielded directly from this generator function, making code easier to read and write.

### Example Usage

Here's a complete example demonstrating how this might work in the context of a generator function:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example list of books
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three"},  # No author
    {"title": "Book Four", "author": "Author A"},  # Duplicate author
]

# Using the generator function
for author in get_authors(books):
    print(author)


### Output


Author A
Author B


### Summary

- The code constructs a set of unique authors from a list of book dictionaries while filtering out entries without an author.
- It yields each unique author from a generator, providing a flexible way to work with potentially large datasets without loading everything into memory at once.

Using `yield from` with a set comprehension is efficient for generating unique values and managing memory usage effectively.

In [23]:
# Get Llama 3.2 to answer
def ask_ollama():
    response = ollama.chat(model=MODEL_LLAMA, messages=messages)
    display(Markdown(response['message']['content']))

In [24]:
ask_ollama()

**Understanding the Code**
==========================

The given code snippet uses a combination of Python features to extract author names from a list of dictionaries (`books`). Here's a breakdown:

* `yield from`: This is a syntax feature introduced in Python 3.3, which allows you to delegate iteration to another iterable.
* `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that filters and extracts values from the `books` list.

**Step-by-Step Explanation**
-----------------------------

1. **Iteration**: The `for book in books` part iterates over each dictionary (`book`) in the `books` list.
2. **Filtering**: The `if book.get("author")` condition filters out dictionaries that don't have an `"author"` key. This ensures we only process dictionaries with author information.
3. **Value Extraction**: Within the iteration, `.get("author")` extracts the value associated with the `"author"` key from each dictionary.

**Yielding Values**
-------------------

The `yield from` keyword is used to delegate the iteration to the generator expression `{book.get("author") for book in books if book.get("author")}`. This means that instead of yielding values directly, we yield values from another iterable (the generator expression).

When we use `yield from`, Python will:

* Start executing the inner code (the generator expression)
* Yield values from the inner iterable
* Pass each yielded value to the outer iteration

In this case, `yield from` allows us to create an iterator that yields author names without having to manually construct a list or other data structure.

**Example Code**
-----------------

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage:
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "publisher": "ABC Inc."},  # No author
    {"title": "Book 3", "author": "Jane Smith"}
]

for author in get_authors(books):
    print(author)  # Output: John Doe, Jane Smith
```

In summary, this code uses a generator expression and `yield from` to efficiently extract author names from a list of dictionaries while filtering out dictionaries without author information.